<a href="https://colab.research.google.com/github/lostcavans/DataScience/blob/main/Copia_de_Untitled0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
url = 'https://www.eldiario.net/portal/category/portada/'
response = requests.get(url)

In [ ]:
response

<Response [200]>

In [ ]:
# Lista para almacenar los datos de los productos
dataNews = []

# Función para extraer los productos de una página
def scrape_page(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        news_items = soup.find('ul', class_='products hover-slider main-products layout-standard columns-4')

        if news_items:
            items = news_items.find_all('li')
            for item in items:
                # Nombre del producto
                nombre = item.find('h2', class_='woocommerce-loop-product__title')
                product_name = nombre.a.text.strip() if nombre and nombre.a else "N/A"

                # URL del producto
                enlace = item.find('div', class_='product-thumbnail').find('a', href=True)
                product_url = enlace['href'] if enlace else "N/A"

                # Imagen del producto
                imagen = item.find('img')
                image_url = imagen['src'] if imagen else "N/A"

                # Precio del producto
                precio = item.find('span', class_='woocommerce-Price-amount')
                product_price = precio.text.strip() if precio else "N/A"

                # Obtener descripción del producto
                product_description = scrape_description(product_url)

                # Guardar en la lista
                dataNews.append([product_name, product_url, image_url, product_price, product_description])

# Función para obtener la descripción de un producto
def scrape_description(product_url):
    try:
        response = requests.get(product_url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            desc_container = soup.find('div', class_='woocommerce-product-details__short-description')
            if desc_container:
                description = desc_container.get_text(strip=True)
                return description
        return "No disponible"
    except Exception as e:
        return f"Error: {e}"

# Obtener todas las páginas disponibles
def get_all_pages(base_url):
    pages = [base_url]  # Lista con la primera página
    response = requests.get(base_url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        pagination = soup.find('ul', class_='page-numbers')

        if pagination:
            links = pagination.find_all('a', class_='page-numbers', href=True)
            for link in links:
                page_url = link['href']
                if page_url not in pages:
                    pages.append(page_url)  # Agregar solo si no está repetida

    return pages

# Obtener todas las URLs de paginación
page_urls = get_all_pages(base_url)

# Scraping de cada página
for page_url in page_urls:
    print(f"Scrapeando: {page_url}")
    scrape_page(page_url)
    time.sleep(1)  # Pequeña pausa para no sobrecargar el servidor

# Crear DataFrame
df = pd.DataFrame(dataNews, columns=["Producto", "URL", "Imagen", "Precio", "Descripción"])

# Configurar pandas para evitar saltos de línea
pd.set_option('display.max_colwidth', None)
pd.set_option('display.expand_frame_repr', False)

# Imprimir el DataFrame
print(df.to_string(index=False, col_space=20))




El Ministerio Público emitirá la resolución de imputación formal en contra del exministro de Medio Ambiente y Agua, Alan Lisperguer, por los delitos de...
El plantel de THe Strongest fue eliminado de la Copa Libertadores al caer anoche jugando de visitante ante el Bahía brasileño por 3 goles...
En coincidencia con su 55 aniversario, la Asociación Nacional de la Prensa (ANP) reeligió a su Directorio presidido por Jorge Carrasco Guzmán para la...
En respuesta a una carta enviada por unas 130 personalidades, el Bloque de Unidad se comprometió, luego de elegir al candidato único, a incluir...
Las perspectivas de la economía nacional no son nada halagadoras parala presente gestión, el análisis de BMI, en “Perspectivas mensuales de América Latina”-Un Enfoque...
El expresidente del Banco Central de Bolivia (BCB), Juan Antonio Morales, manifestó ayer que debería existir mayor prudencia en la emisión monetaria, debido a...
Choferes del transporte público en La Paz, con insultos, golpes y patada

In [ ]:
dataNews

[{'title': 'Imputarán a Lisperguer  por enriquecimiento ilícito',
  'link': 'https://www.eldiario.net/portal/2025/02/26/imputaran-a-lisperguer-por-enriquecimiento-ilicito/',
  'linkImage': 'https://www.eldiario.net/portal/wp-content/uploads/2025/02/P-5-9-696x464.jpg',
  'textNew': 'El Ministerio Público emitirá la resolución de imputación formal en contra del exministro de Medio Ambiente y Agua, Alan Lisperguer, por los delitos de...'},
 {'title': 'The Strongest eliminado de la Libertadores',
  'link': 'https://www.eldiario.net/portal/2025/02/26/the-strongest-eliminado-de-la-libertadores/',
  'linkImage': 'https://www.eldiario.net/portal/wp-content/uploads/2025/02/P-4-8-696x290.jpg',
  'textNew': 'El plantel de THe Strongest fue eliminado de la Copa Libertadores al caer anoche jugando de visitante ante el Bahía brasileño por 3 goles...'},
 {'title': 'ANP reelige a su Directorio presidido  por Jorge Carrasco Guzmán',
  'link': 'https://www.eldiario.net/portal/2025/02/25/anp-reelige-a-su

In [ ]:
df = pd.DataFrame(dataNews)

In [ ]:
df

,title,link,linkImage,textNew
0,Imputarán a Lisperguer por enriquecimiento il...,https://www.eldiario.net/portal/2025/02/26/imp...,https://www.eldiario.net/portal/wp-content/upl...,El Ministerio Público emitirá la resolución de...
1,The Strongest eliminado de la Libertadores,https://www.eldiario.net/portal/2025/02/26/the...,https://www.eldiario.net/portal/wp-content/upl...,El plantel de THe Strongest fue eliminado de l...
2,ANP reelige a su Directorio presidido por Jor...,https://www.eldiario.net/portal/2025/02/25/anp...,https://www.eldiario.net/portal/wp-content/upl...,"En coincidencia con su 55 aniversario, la Asoc..."
3,Bloque de Unidad se compromete a incluir en s...,https://www.eldiario.net/portal/2025/02/25/blo...,https://www.eldiario.net/portal/wp-content/upl...,En respuesta a una carta enviada por unas 130 ...
4,"Bolivia registrará un crecimiento de 0,9%, un...",https://www.eldiario.net/portal/2025/02/25/bol...,https://www.eldiario.net/portal/wp-content/upl...,Las perspectivas de la economía nacional no so...
5,Sugieren más prudencia en la emisión monetaria,https://www.eldiario.net/portal/2025/02/25/sug...,https://www.eldiario.net/portal/wp-content/upl...,El expresidente del Banco Central de Bolivia (...
6,"Con insultos, golpes y patadas choferes preten...",https://www.eldiario.net/portal/2025/02/25/con...,https://www.eldiario.net/portal/wp-content/upl...,"Choferes del transporte público en La Paz, con..."
7,Hay filas por combustible en ciudades del eje ...,https://www.eldiario.net/portal/2025/02/25/hay...,https://www.eldiario.net/portal/wp-content/upl...,En los últimos días se ha podido observar nuev...
8,Fallecidos en accidentes viales suman 77 en do...,https://www.eldiario.net/portal/2025/02/25/fal...,https://www.eldiario.net/portal/wp-content/upl...,Un total de 77 personas fallecieron en acciden...
9,El Papa experimenta una “ligera mejoría”,https://www.eldiario.net/portal/2025/02/25/el-...,https://www.eldiario.net/portal/wp-content/upl...,"El Papa Francisco ha experimentado una ""ligera..."


In [ ]:
news_items

<div class="td_block_inner tdb-block-inner td-fix-index" id="tdi_58">
<div class="tdb_module_loop td_module_wrap td-animation-stack td-cpt-post">
<div class="td-module-container td-category-pos-image">
<div class="td-image-container">
<a class="td-post-category" href="https://www.eldiario.net/portal/category/portada/">Portada</a> <div class="td-module-thumb"><a class="td-image-wrap" href="https://www.eldiario.net/portal/2025/02/26/imputaran-a-lisperguer-por-enriquecimiento-ilicito/" rel="bookmark" title="Imputarán a Lisperguer  por enriquecimiento ilícito"><span class="entry-thumb td-thumb-css" data-img-url="https://www.eldiario.net/portal/wp-content/uploads/2025/02/P-5-9-696x464.jpg" data-type="css_image"></span></a></div> </div>
<div class="td-module-meta-info">
<h3 class="entry-title td-module-title"><a href="https://www.eldiario.net/portal/2025/02/26/imputaran-a-lisperguer-por-enriquecimiento-ilicito/" rel="bookmark" title="Imputarán a Lisperguer  por enriquecimiento ilícito">Imput